In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Load the dataset using Keras ImageDataGenerator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/imgess/images',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# Train the model
model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    verbose=1)



Found 7508 images belonging to 4 classes.
Epoch 1/10
235/235 [==============================] - 141s 534ms/step - loss: 0.4631 - accuracy: 0.8347
Epoch 2/10
235/235 [==============================] - 101s 431ms/step - loss: 0.2204 - accuracy: 0.9233
Epoch 3/10
235/235 [==============================] - 99s 419ms/step - loss: 0.1475 - accuracy: 0.9487
Epoch 4/10
235/235 [==============================] - 98s 418ms/step - loss: 0.1203 - accuracy: 0.9575
Epoch 5/10
235/235 [==============================] - 105s 446ms/step - loss: 0.0843 - accuracy: 0.9724
Epoch 6/10
235/235 [==============================] - 101s 430ms/step - loss: 0.0763 - accuracy: 0.9739
Epoch 7/10
235/235 [==============================] - 99s 423ms/step - loss: 0.0565 - accuracy: 0.9823
Epoch 8/10
235/235 [==============================] - 99s 421ms/step - loss: 0.0526 - accuracy: 0.9830
Epoch 9/10
235/235 [==============================] - 98s 418ms/step - loss: 0.0406 - accuracy: 0.9872
Epoch 10/10
235/235 [======

In [1]:
# Save the trained model
# model.save('/kaggle/working/clothing_modelCnnV1.h5')


# Testing CNN

In [13]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the image
img = image.load_img('/kaggle/input/testimg1/clothing_image.jpg', target_size=(224, 224))

# Convert the image to a numpy array
img = np.array(img)

# Normalize the pixel values
img = img.astype('float32') / 255.0

# Add an extra dimension to the array to represent batch size
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))

# Make a prediction on the image
probabilities = model.predict(img)

# Get the class label with the highest probability
class_index = np.argmax(probabilities)

# Get the corresponding class name
class_names = ['pants', 'shirts', 'shorts', 'tshirt']
class_name = class_names[class_index]

print('The predicted class is:', class_name)


1/1 [==============================] - 0s 335ms/step
The predicted class is: shirts


# Pose Estimation

In [3]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import mediapipe as mp
import os
import time
holistics = mp.solutions.holistic # To bring our holistic model
drawing = mp.solutions.drawing_utils # Use fot drawing the utilities

def Detection(img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    img.flags.writeable = False                  # Image is no longer writeable
    results = model.process(img)                 # Make prediction
    img.flags.writeable = True                   # Image is now writeable 
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return img, results
def DrawingCustomLandmarks(image, results):
    # Draw face connections
    drawing.draw_landmarks(image, results.face_landmarks, holistics.FACEMESH_TESSELATION, 
                             drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    drawing.draw_landmarks(image, results.pose_landmarks, holistics.POSE_CONNECTIONS,
                             drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    drawing.draw_landmarks(image, results.left_hand_landmarks, holistics.HAND_CONNECTIONS, 
                             drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    drawing.draw_landmarks(image, results.right_hand_landmarks, holistics.HAND_CONNECTIONS, 
                             drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
    
    
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with holistics.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = Detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        DrawingCustomLandmarks(image, results)

        # Show to screen
        cv2.imshow('Pose Estimation Window', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# Object Detection

In [35]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the YOLOv4-tiny object detection model
net = cv2.dnn.readNetFromDarknet("yolov4-tiny.cfg", "yolov4-tiny.weights")

# Define the classes that the model can detect
classes = ["shirt", "pants", "tshirt", "shorts"]

# Initialize the video capture
cap = cv2.VideoCapture(0)

while True:
    # Read frames from the video capture
    ret, frame = cap.read()

    # Perform object detection on the frame
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416,416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in [net.getUnconnectedOutLayers()][0]]
    outputs = net.forward(output_layers)

    # Process the output of the object detection model
    class_ids = []
    confidences = []
    boxes = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                width = int(detection[2] * frame.shape[1])
                height = int(detection[3] * frame.shape[0])
                left = int(center_x - width/2)
                top = int(center_y - height/2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Draw bounding boxes around the detected objects
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    if len(indices) > 0:
        for i in indices.flatten():
            left, top, width, height = boxes[i]
            label = classes[class_ids[i]]
            confidence = confidences[i]
            cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow("Object Detection", frame)

    # Check for key presses and exit gracefully if necessary
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and close the window
cap.release()
cv2.destroyAllWindows()


# Combining the object detection and pose estimation model

In [ ]:
import cv2
import mediapipe as mp
model_path = 'yolov4-tiny.weights'
config_path = 'yolov4-tiny.cfg'
net = cv2.dnn.readNetFromCaffe(config_path, model_path)

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
blob = cv2.dnn.blobFromImage(frame, 0.007843, (frame.shape[1], frame.shape[0]), (127.5, 127.5, 127.5), False)
net.setInput(blob)
detections = net.forward()
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.5:
        class_id = int(detections[0, 0, i, 1])
        if class_id == 1: # Shirt
            # Perform pose estimation on the upper body
            # (you may need to adjust the coordinates based on the output of your object detection model)
            upper_body = frame[y:y+h, x:x+w]
            results = pose.process(cv2.cvtColor(upper_body, cv2.COLOR_BGR2RGB))
            
            # Get the pose landmarks and estimate the torso angle
            landmarks = results.pose_landmarks.landmark
            torso_angle = # Estimate the torso angle using the pose landmarks
            
            # Fit the shirt to the torso using the estimated angle
            # (you may need to adjust the coordinates and angle based on your shirt and torso models)
            shirt = # Load the shirt model
            shirt.fit_to_torso(upper_body, landmarks, torso_angle)
cv2.imshow('Frame', frame)
cv2.waitKey(1)


In [83]:
import os

import cvzone
import cv2
from cvzone.PoseModule import PoseDetector

# cap = cv2.VideoCapture("Resources/Videos/1.mp4")
cap = cv2.VideoCapture(0)
detector = PoseDetector()
cap.set(3, 1280)  # Set the width of the frame to 1280
cap.set(4, 720)   # Set the height of the frame to 720


shirtFolderPath = "Resources/Shirts"
listShirts = os.listdir(shirtFolderPath)
# print(listShirts)
fixedRatio = 262 / 190  
shirtRatioHeightWidth = 581 / 440
imageNumber = 0
imgButtonRight = cv2.imread("Resources/button.png", cv2.IMREAD_UNCHANGED)
imgButtonLeft = cv2.flip(imgButtonRight, 1)
counterRight = 0
counterLeft = 0
selectionSpeed = 10

while True:
    success, img = cap.read()
    img = detector.findPose(img)
    # img = cv2.flip(img,1)
    lmList, bboxInfo = detector.findPosition(img, bboxWithHands=False, draw=False)
    if lmList:
        # center = bboxInfo["center"]
        lm11 = lmList[11][1:3]
        lm12 = lmList[12][1:3]
        imgShirt = cv2.imread(os.path.join(shirtFolderPath, listShirts[imageNumber]), cv2.IMREAD_UNCHANGED)

        widthOfShirt = int((lm11[0] - lm12[0]) * fixedRatio)
        print(widthOfShirt)
        imgShirt = cv2.resize(imgShirt, (widthOfShirt, int(widthOfShirt * shirtRatioHeightWidth)))
        currentScale = (lm11[0] - lm12[0]) / 190
        offset = int(44 * currentScale), int(48 * currentScale)

        try:
            img = cvzone.overlayPNG(img, imgShirt, (lm12[0] - offset[0], lm12[1] - offset[1]))
        except:
            pass

#         img = cvzone.overlayPNG(img, imgButtonRight, (1074, 293))
#         img = cvzone.overlayPNG(img, imgButtonLeft, (72, 293))

#         if lmList[16][1] < 300:
#             counterRight += 1
#             cv2.ellipse(img, (139, 360), (66, 66), 0, 0,
#                         counterRight * selectionSpeed, (0, 255, 0), 20)
#             if counterRight * selectionSpeed > 360:
#                 counterRight = 0
#                 if imageNumber < len(listShirts) - 1:
#                     imageNumber += 1
#         elif lmList[15][1] > 900:
#             counterLeft += 1
#             cv2.ellipse(img, (1138, 360), (66, 66), 0, 0,
#                         counterLeft * selectionSpeed, (0, 255, 0), 20)
#             if counterLeft * selectionSpeed > 360:
#                 counterLeft = 0
#                 if imageNumber > 0:
#                     imageNumber -= 1

#         else:
#             counterRight = 0
#             counterLeft = 0

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

68
71
70
73
75
75
74
74
74
74
74
74
74
75
75
75
74
74
74
74
74
75
77
75
77
78
78
79
77
78
71


In [95]:
cap.release()
cv2.destroyAllWindows()


In [91]:
!streamlit run Clothapp.py

^C


In [25]:
import os

import cvzone
import cv2
from cvzone.PoseModule import PoseDetector

# cap = cv2.VideoCapture("Resources/Videos/1.mp4")
cap = cv2.VideoCapture(0)
detector = PoseDetector()
cap.set(3, 1280)  # Set the width of the frame to 1280
cap.set(4, 720)   # Set the height of the frame to 720


shirtFolderPath = "Resources/Shirts"
listShirts = os.listdir(shirtFolderPath)
# print(listShirts)
fixedRatio = 262 / 190  
shirtRatioHeightWidth = 581 / 440
imageNumber = 0
counterRight = 0
counterLeft = 0
selectionSpeed = 10

while True:
    success, img = cap.read()
    img = detector.findPose(img)
    # img = cv2.flip(img,1)
    lmList, bboxInfo = detector.findPosition(img, bboxWithHands=False, draw=False)
    if lmList:
        # center = bboxInfo["center"]
        lm11 = lmList[11][1:3]
        lm12 = lmList[12][1:3]
        imgShirt = cv2.imread(os.path.join(shirtFolderPath, listShirts[imageNumber]), cv2.IMREAD_UNCHANGED)

        widthOfShirt = int((lm11[0] - lm12[0]) * fixedRatio)
        print(widthOfShirt)
        imgShirt = cv2.resize(imgShirt, (widthOfShirt, int(widthOfShirt * shirtRatioHeightWidth)))
        currentScale = (lm11[0] - lm12[0]) / 190
        offset = int(44 * currentScale), int(48 * currentScale)

        try:
            img = cvzone.overlayPNG(img, imgShirt, (lm12[0] - offset[0], lm12[1] - offset[1]))
        except:
            pass

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

445
438
431
420
415
417
420
421
421
424
428
430
435
439
435
434
434
435
437
437
437
435
437
437
435
435
432
432
434
434
434
434
434
437
439
441
442
442
442
441
439
441
442
442
445
445
448
449
449
449
446
445
445
444
442
441
439
437
437
437
437
439
439
439
439
441
438
438
438
437
439
441
420
401
405
406
408
408
408
409
427
432
432
435
435
437
437
439
441
441
442
442
441
442
442
442
445
445


In [24]:
cap.release()
cv2.destroyAllWindows()